# Proyecto C19 Python

In [23]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
import os
import glob

In [24]:
#Define dataset de examenes PCR por region/fecha
pcrStdDf = pd.read_csv(os.getcwd() + "/dataSets/producto7/PCR_std.csv", index_col=None, header=0)
pcrStdDf.columns = ['region', 'codigo_region', 'poblacion', 'fecha', 'numero_pcr']

#Define dataset de casos nuevos por region/fecha
casosNuevosDf = pd.read_csv(os.getcwd() + "/dataSets/producto13/CasosNuevosCumulativo_std.csv", index_col=None, header=0)
casosNuevosDf.columns = ['region', 'fecha', 'total']
#ToDo verificar el tema del codigo de region para casosNuevosDf
#https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

In [25]:
casosNuevosDfCondicionesCodRegion = (
    (casosNuevosDf['region'] == 'Arica y Parinacota'),
    (casosNuevosDf['region'] == 'Tarapacá'),
    (casosNuevosDf['region'] == 'Antofagasta'),
    (casosNuevosDf['region'] == 'Atacama'),
    (casosNuevosDf['region'] == 'Coquimbo'),
    (casosNuevosDf['region'] == 'Valparaíso'),
    (casosNuevosDf['region'] == 'O’Higgins'),
    (casosNuevosDf['region'] == 'Maule'),
    (casosNuevosDf['region'] == 'Ñuble'),
    (casosNuevosDf['region'] == 'Biobío'),
    (casosNuevosDf['region'] == 'Araucanía'),
    (casosNuevosDf['region'] == 'Los Ríos'),
    (casosNuevosDf['region'] == 'Los Lagos'),
    (casosNuevosDf['region'] == 'Aysén'),
    (casosNuevosDf['region'] == 'Magallanes'),
    (casosNuevosDf['region'] == 'Metropolitana')
)

casosNuevosDfValoresCodRegion = [15,1,2,3,4,5,6,7,16,8,9,14,10,11,12,13]

casosNuevosDf['codigo_region'] = np.select(casosNuevosDfCondicionesCodRegion, casosNuevosDfValoresCodRegion)

#casosNuevosDf.head(17)

In [34]:
#Define dataframes product2/4 en un mapa, y cada key es una fecha de un archivo encontrado
def loadDataFrameToMap(carpeta, indexKey) :
        
    files = [os.path.basename(x) for x in glob.glob(carpeta)]
    
    mapDataFramesByDate = {}

    for fileName in files:
        keyMap = int((fileName[0: -indexKey]).replace("-", ''))
        mapDataFramesByDate[keyMap] = pd.read_csv(carpeta[0: -5] + fileName, index_col=None, header=0)
    
    #ToDo: ordenar por orden ascendente(?)
    return pd.Series(mapDataFramesByDate).sort_index()

In [35]:
pathProduct2 = os.getcwd() + "/dataSets/producto2/*.csv";
product2MapDF = loadDataFrameToMap(pathProduct2, 21)
#product2MapDF[20210201]

pathProduct4 = os.getcwd() + "/dataSets/producto4/*.csv";
product4MapDF = loadDataFrameToMap(pathProduct4, 35)
#product4MapDF[20210607]

In [36]:
#ToDo: verificar si es necesaria o no esta funcion!!!!
def obtainDataFrames(mapDataFrames, fechaDesde, fechaHasta, exclusiveDesde = False, exclusiveHasta = False) :
    
    if (fechaHasta < fechaDesde) :
        raise Exception('La fecha de la variable fechaHasta, no puede ser menor a la variable fechaDesde') 
    
    if (exclusiveDesde) :
        fechaDesde = fechaDesde + 1
        
    if (exclusiveHasta) :
        fechaHasta = fechaHasta - 1
        
    mapFechasEntre = {}
    for mapKeyFecha in mapDataFrames.keys() :
        if(mapKeyFecha >= fechaDesde and mapKeyFecha <= fechaHasta) :
            mapFechasEntre[mapKeyFecha] = mapDataFrames[mapKeyFecha]

    return pd.Series(mapFechasEntre).sort_index()


obtainDataFrames(product4MapDF, 20210101, 20210105, False, False)


20210101                               Region  Casos total...
20210102                               Region  Casos total...
20210103                               Region  Casos total...
20210104                               Region  Casos total...
20210105                               Region  Casos total...
dtype: object

# Positividad de casos

(Proporcion de test positivos por dia / Total de test efectuados en un dia) * 100

- Proporcion de test positivos por dia => DataSet product4 col: "casos nuevos totales"
- Total de test efectuados en un dia => DataSet product7 col: "numero"

In [ ]:
def obtenerPcrPorFechaYDistrito(fecha, distrito) :
    predicadoFecha = pcrStdDf["fecha"] == fecha
    predicadoDistrito = pcrStdDf["codigo_region"] == distrito
    
    return pcrStdDf[
        (predicadoDistrito) & 
        (predicadoFecha)
    ]

#obtenerPcrPorFechaYDistrito('2021-08-09', 5)

In [40]:
def obtenerCasosNuevosPorFechaYDistrito(fecha, distrito) :
    predicadoFecha = casosNuevosDf["fecha"] == fecha
    predicadoDistrito = pd.to_numeric(casosNuevosDf["codigo_region"]) == distrito
    
    return casosNuevosDf[
        (predicadoFecha) & 
        (predicadoDistrito)
    ]

#obtenerCasosNuevosPorFechaYDistrito('2021-08-09', 5)

In [41]:
def calculoUmbral(positividad):
    if(positividad <= 3) :
        return "verde"
    elif (positividad <= 5 & positividad > 3) :
        return "amarillo"
    elif (positividad > 5) :
        return "naranja"

def calcularPositividadYUmbral(fecha, distrito) :
    casosNuevos = obtenerCasosNuevosPorFechaYDistrito(fecha, distrito)
    pcr = obtenerPcrPorFechaYDistrito(fecha, distrito)

    positividad = ((casosNuevos['total'].item() / pcr['numero_pcr'].item()) * 100)
    
    return pd.Series({
        "positividad" : positividad, 
        "umbral": calculoUmbral(positividad)
    })

#calcularPositividadYUmbral('2021-08-09', 5)